In [ ]:
!pip install opencv-python matplotlib

In [ ]:
import sys
sys.path.append(r'c:\ai_project01\Pytorch_Retinaface')

In [ ]:
import torch
import cv2
import numpy as np
from models.retinaface import RetinaFace
from data import cfg_re50
from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
from utils.box_utils import decode
torch.cuda.is_available()

In [ ]:
device = "cuda"
cfg = cfg_re50
net = RetinaFace(cfg=cfg, phase='test').to(device)
pretrained_path=r'C:\ai_project01\Pytorch_Retinaface\weights\Resnet50_Final.pth'
state_dict = torch.load(pretrained_path, map_location=device, weights_only=True)
new_state_dict={}
for k,v in state_dict.items():
    if k.startswith("module."):
        new_state_dict[k[7:]] = v
    else:
        new_state_dict[k] = v
net.load_state_dict(new_state_dict, strict = True)

In [ ]:
net.eval()
image_path = r'C:\ai_project01\mask_images\no_mask\no_mask_00000.png'
img_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)
img = np.float32(img_raw)
im_height, im_width, _ = img.shape
scale = torch.Tensor([im_width, im_height, im_width, im_height]).to(device)
img -= (104, 117, 123)
img = img.transpose(2,0,1)
img = torch.from_numpy(img).unsqueeze(0).to(device)
loc, conf, landms = net(img)
priorbox = PriorBox(cfg, image_size = (im_height, im_width))
priors = priorbox.forward().to(device)
prior_data = priors.data
boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
boxes = boxes * scale
scores = conf.squeeze(0).data.cpu().numpy()[:,1]
confidence_threshold = 0.5
top_indices = np.where(scores>confidence_threshold)[0]
boxes = boxes[top_indices]
scores = scores[top_indices]
dets = np.hstack((boxes.cpu().numpy(), scores[:, np.newaxis])).astype(np.float32, copy=False)
keep = py_cpu_nms(dets, 0.3)
dets = dets[keep, :]

In [ ]:
for b in dets:
    if b[4] < confidence_threshold:
        continue

    text = "{:.2f}".format(b[4])
    b=list(map(int,b))
    cv2.rectangle(img_raw, (b[0], b[1]), (b[2], b[3]), (0,255,0), 2)
    cx, cy = b[0], b[1] + 12
    cv2.putText(img_raw, text, (cx,cy), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255,255,255))
cv2.imshow('RetinaFace PyTorch Detection', img_raw)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import time
import os

folder_path = r'C:\ai_project01\mask_images\mask_on'
image_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.png')])

while True:
    for file_name in image_files:
        image_path = os.path.join(folder_path, file_name)
        img_raw = cv2.imread(image_path)
        if img_raw is None:
            continue

        # 전처리 및 얼굴 검출
        img = np.float32(img_raw)
        im_height, im_width, _ = img.shape
        scale = torch.Tensor([im_width, im_height, im_width, im_height]).to(device)
        img -= (104, 117, 123)
        img = img.transpose(2,0,1)
        img = torch.from_numpy(img).unsqueeze(0).to(device)

        with torch.no_grad():
            loc, conf, landms = net(img)

        priorbox = PriorBox(cfg, image_size=(im_height, im_width))
        priors = priorbox.forward().to(device)
        prior_data = priors.data
        boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])

        scores = conf.squeeze(0).data.cpu().numpy()[:,1]
        confidence_threshold = 0.5
        top_indices = np.where(scores > confidence_threshold)[0]

        boxes = boxes[top_indices] * scale
        boxes = boxes.cpu().numpy()
        scores = scores[top_indices]

        dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
        keep = py_cpu_nms(dets, 0.3)
        dets = dets[keep, :]

        for b in dets:
            if b[4] < confidence_threshold:
                continue
            text = "{:.2f}".format(b[4])
            b = list(map(int, b))
            cv2.rectangle(img_raw, (b[0], b[1]), (b[2], b[3]), (0,255,0), 2)
            cv2.putText(img_raw, text, (b[0], b[1]+12), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255,255,255))

        cv2.imshow('RetinaFace Detection', img_raw)
        key = cv2.waitKey(500) & 0xFF  # 1초 대기

        if key == ord('q'):
            cv2.destroyAllWindows()
            break
    else:
        continue  # 안쪽 for문 정상 종료 → 다시 처음부터 반복
    break  # q 눌러서 종료했을 때 while도 빠져나감

